In [99]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime,timezone

from dotenv import dotenv_values

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account

from dotenv import dotenv_values
# https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries
# https://codelabs.developers.google.com/codelabs/cloud-bigquery-python#0

# https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# https://cloud.google.com/python/docs/setup
# https://cloud.google.com/apis/docs/getting-started#enabling_apis



# https://cloud.google.com/bigquery/docs/reference/libraries
# https://cloud.google.com/python/docs/reference/bigquery/latest
# https://gcloud.readthedocs.io/en/latest/bigquery-client.html


In [100]:
# credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\xxxxxx.json')
# print(credentials.project_id)

In [101]:
start_date_query='2020-01-01'

projectId='pongthorn'  # smart-data-ml
dataset_id='SMartDW'
table_id = f"{projectId}.{dataset_id}.incident"

env_path='.env'
config = dotenv_values(dotenv_path=env_path)

In [102]:
# client = bigquery.Client(credentials= credentials,project=projectId)
client = bigquery.Client(project=projectId)

In [103]:
try:
    table=client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
    print(table.schema)
except NotFound:
    raise Exception("Table {} is not found.".format(table_id))

Table pongthorn.SMartDW.incident already exists.
[SchemaField('id', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('incident_no', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('severity_id', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('severity_name', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('sla', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('product_type', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('brand', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('model', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('incident_type', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('status', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('service_type', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('is_failure_type', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('count_detail', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('open_datetime', 'DAT

In [104]:
dt_imported=datetime.now(timezone.utc) # utc
#dt_imported=datetime.now() # utc
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Imported DateTime: {str_imported}" )

Imported DateTime: 2023-07-19 17:22:21


In [105]:
sql_lastImport=f"SELECT max(imported_at) as last_imported from `{table_id}` where open_datetime>='{start_date_query}' "
job_lastImported=client.query(sql_lastImport)
str_lastImported=None
for row in job_lastImported:    
    if row.last_imported is not None: 
        str_lastImported=row.last_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Last Imported DateTime: {str_lastImported}" )

if str_lastImported is not None:
  start_date_query=str_lastImported

print(f"Start Import on update_at of last imported date : {start_date_query}" )

Last Imported DateTime: 2023-07-19 17:19:08
Start Import on update_at of last imported date : 2023-07-19 17:19:08


In [106]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [107]:
# check schema 
#,timezone('UTC', incident.updated_at) as updated_at_utc
# and incident.updated_at>=%(start_date_param)s
sql_incident="""

select
incident.id as id, incident.incident_no as incident_no,

severity.id as  severity_id,
severity.severity_name as  severity_name,

service_level.sla_name as sla,

product_type.productype_name as product_type,brand.brand_name as brand,

xtype.incident_type_name as incident_type,
status.incident_status_name as status,
service.service_type_name service_type,

model.model_name as model,CASE WHEN failure_type IS NULL THEN  0 ELSE 1 END AS is_failure_type,


(select count(*) from  app_incident_detail  as detail where  detail.incident_master_id=incident.id ) as count_detail


,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as close_datetime

,TO_CHAR(incident.incident_problem_start  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as response_datetime
,TO_CHAR(incident.incident_problem_end  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as resolved_datetime

,company.company_name as company
,TO_CHAR(incident.updated_at,'YYYY-MM-DD HH24:MI:SS') as updated_at 

from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id
inner join  app_incident_status as status on incident.incident_status_id = status.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join  app_service_type as service on incident.service_type_id= service.id

inner join app_inventory as inventory on incident.inventory_id = inventory.id

inner join app_brand as brand on inventory.brand_id = brand.id
inner join app_model as model on inventory.model_id = model.id
inner join app_product_type as product_type on inventory.product_type_id = product_type.id
inner join app_sla as service_level on inventory.customer_sla_id = service_level.id

inner join app_project as project on inventory.project_id = project.id
inner join app_company as company on project.company_id = company.id

where incident.incident_status_id =4
and timezone('UTC', incident.updated_at)>=%(start_date_param)s

order by incident.updated_at desc

"""
print(sql_incident)



select
incident.id as id, incident.incident_no as incident_no,

severity.id as  severity_id,
severity.severity_name as  severity_name,

service_level.sla_name as sla,

product_type.productype_name as product_type,brand.brand_name as brand,

xtype.incident_type_name as incident_type,
status.incident_status_name as status,
service.service_type_name service_type,

model.model_name as model,CASE WHEN failure_type IS NULL THEN  0 ELSE 1 END AS is_failure_type,


(select count(*) from  app_incident_detail  as detail where  detail.incident_master_id=incident.id ) as count_detail


,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as close_datetime

,TO_CHAR(incident.incident_problem_start  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as response_datetime
,TO_CHAR(incident.incident_problem_end  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI'

In [108]:
print("Create all issues dataframe")

dict_params={"start_date_param":start_date_query}
df_all=list_data(sql_incident,dict_params,get_postgres_conn())

if df_all.empty==True:
    print("no transsaction update")
    exit()
else:
    print("import data to bigquery")

df_all.head()

Create all issues dataframe
no transsaction update


""


In [95]:
# convert object to datetime
dateTimeCols=['open_datetime','response_datetime','resolved_datetime','close_datetime']
for col in dateTimeCols:
 df_all[col]=pd.to_datetime(df_all[col], format='%Y-%m-%d %H:%M',errors= 'coerce')

dateTimeCols2=['updated_at']
for col in dateTimeCols2:
 df_all[col]=pd.to_datetime(df_all[col], format='%Y-%m-%d %H:%M:%S',errors= 'coerce')

# UTC Import
df_all['imported_at']=dt_imported

In [96]:
df_all.dropna(inplace=True)

#df_all=df_all.head(10)
print(df_all.info())
df_all.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   id                 2 non-null      int64              
 1   incident_no        2 non-null      object             
 2   severity_id        2 non-null      int64              
 3   severity_name      2 non-null      object             
 4   sla                2 non-null      object             
 5   product_type       2 non-null      object             
 6   brand              2 non-null      object             
 7   incident_type      2 non-null      object             
 8   status             2 non-null      object             
 9   service_type       2 non-null      object             
 10  model              2 non-null      object             
 11  is_failure_type    2 non-null      int64              
 12  count_detail       2 non-null      int64              

,id,incident_no,severity_id,severity_name,sla,product_type,brand,incident_type,status,service_type,model,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company,updated_at,imported_at
0,3198,SR-ES-23-3198,1,Critical,24x7 6Hrs Resolution Time,Software,YIP,General Incident,Closed,Incident,Incident System Management,0,1,2023-07-20 00:00:00,2023-07-20 00:16:00,2023-07-20 00:05:00,2023-07-20 00:15:00,YIP,2023-07-20 00:18:12,2023-07-19 17:19:08.368414+00:00
1,3195,SR-ES-23-3195,2,Major,24x7 6Hrs Resolution Time,Software,YIP,General Incident,Closed,Incident,Incident System Management,0,1,2023-07-10 09:00:00,2023-07-19 23:10:00,2023-07-19 10:00:00,2023-07-19 22:00:00,YIP,2023-07-20 00:15:22,2023-07-19 17:19:08.368414+00:00


In [97]:
def insertDataFrameToBQ(df_trasns):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            df_trasns, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_trasns), "Imported closed incident to bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        print(e) 

try:
    insertDataFrameToBQ(df_all)
except Exception as ex:
    raise ex




Total  2 Imported closed incident to bigquery successfully


In [98]:
importDate=dt_imported.strftime("%Y-%m-%d")

def load_data_bq(sql:str):
 query_result=client.query(sql)
 dfx=query_result.to_dataframe()
 return dfx
sql_query=f""" SELECT * FROM `{table_id}` WHERE imported_at >= '{importDate}' LIMIT 10 """
df=load_data_bq(sql_query)
df.head()

,id,incident_no,severity_id,severity_name,sla,product_type,brand,model,incident_type,status,service_type,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company,updated_at,imported_at
0,2638,SR-ES-23-2638,1,Critical,24x7 6Hrs Response Time,Server,HPE,StoreOnce 3640 48TB System,Backup Failure,Closed,Incident,0,14,2023-04-16 20:38:00,2023-05-05 11:52:00,2023-04-16 20:41:00,2023-05-05 11:52:00,IBANK,2023-07-14 11:22:05,2023-07-19 16:59:57.644021
1,2976,SR-ES-23-2976,2,Major,24x7 4Hrs Response Time,Storage,NetApp,SG5712,Controller/Node Failure,Closed,Incident,0,3,2023-06-16 09:12:00,2023-06-16 13:03:00,2023-06-16 09:12:00,2023-06-16 13:00:00,AIS,2023-07-12 15:25:40,2023-07-19 16:59:57.644021
2,2927,SR-ES-23-2927,2,Major,24x7 4Hrs Response Time,Storage,NetApp,SG5712,Controller/Node Failure,Closed,Incident,0,2,2023-06-09 02:28:00,2023-06-09 19:51:00,2023-06-09 14:34:00,2023-06-09 19:51:00,AIS,2023-07-12 16:19:25,2023-07-19 16:59:57.644021
3,2935,SR-ES-23-2935,2,Major,24x7 4Hrs Response Time,Storage,NetApp,FAS8020,Hard Disk Drive Failure,Closed,Incident,1,2,2023-06-09 10:06:00,2023-06-09 18:03:00,2023-06-09 10:36:00,2023-06-09 18:03:00,AIS,2023-07-10 15:36:35,2023-07-19 16:59:57.644021
4,2994,SR-ES-23-2994,2,Major,24x7 4Hrs Resolution Time,Storage,NetApp,FAS8300,Network Cable Failure,Closed,Incident,0,7,2023-06-20 13:20:00,2023-07-10 17:50:00,2023-06-20 13:30:00,2023-06-20 13:30:00,KTBCS,2023-07-10 17:53:09,2023-07-19 16:59:57.644021
